In [1]:
!apt-get install -y tesseract-ocr
!apt-get install -y imagemagick
!apt install tesseract-ocr-rus -y
!pip install pandas tqdm

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts
  imagemagick-6-common imagemagick-6.q16 libdjvulibre-text libdjvulibre21
  libfftw3-double3 libgs9 libgs9-common libidn12 libijs-0.35 libjbig2dec0
  libjxr-tools libjxr0 liblqr-1-0 libmagickcore-6.q16-6
  libmagickcore-6.q16-6-extra libmagickwand-6.q16-6 libnetpbm10
  libwmflite-0.2-7 netpbm poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre
  ghostscript-x imagemagick-doc autotrace cups-bsd | lpr | lprng enscript gimp
  gnuplot grads hp2xx html2ps libwmf-bin mplayer povray radiance sane-utils
 

In [3]:
!git clone https://github.com/tesseract-ocr/tesstrain.git
!mkdir -p tessdata_best
!wget -P tessdata_best https://github.com/tesseract-ocr/tessdata_best/raw/main/rus.traineddata

fatal: destination path 'tesstrain' already exists and is not an empty directory.
--2025-05-15 07:35:38--  https://github.com/tesseract-ocr/tessdata_best/raw/main/rus.traineddata
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tesseract-ocr/tessdata_best/main/rus.traineddata [following]
--2025-05-15 07:35:38--  https://raw.githubusercontent.com/tesseract-ocr/tessdata_best/main/rus.traineddata
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15301764 (15M) [application/octet-stream]
Saving to: ‘tessdata_best/rus.traineddata.1’

rus.traineddata.1   100%[===================>]  14.59M  --.-KB/s  

In [4]:
from google.colab import drive
import zipfile
import os

drive.mount('/content/drive')

archive_path = "/content/drive/MyDrive/part_1.zip"
extract_path = "/content/data"

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(archive_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("📁 Архив распакован.")

Mounted at /content/drive
📁 Архив распакован.


In [5]:
import os
import pandas as pd

csv_path = "/content/data/part_1/all_files_final.csv"
df = pd.read_csv(csv_path)
def fix_path(path):
    full_path = path
    if full_path[:1] == ".":
        full_path = "/content/data" + path[1:]
    if "'" in full_path:
        fixed = full_path.replace("'", "_")
        try:
            os.rename(full_path, fixed)
        except FileNotFoundError:
            pass
        return fixed
    return full_path
df["new_path"] = df["new_path"].apply(fix_path)
df.to_csv(csv_path, index=False)

In [6]:
import pandas as pd
from PIL import Image
import os
import subprocess

!rm -rf /content/data/part_1_gt
output_dir = "/content/data/part_1_gt"
os.makedirs(output_dir, exist_ok=True)
df = pd.read_csv("/content/data/part_1/all_files_final.csv").head(10000)
for _, row in df.iterrows():
    img_path = row["new_path"]
    word = str(row["word_true"])

    if not os.path.isfile(img_path):
        print(f"Не найден файл: {img_path}")
        continue

    filename_base = os.path.splitext(os.path.basename(img_path))[0]
    new_img_path = os.path.join(output_dir, filename_base + ".tif")
    gt_path = os.path.join(output_dir, filename_base + ".gt.txt")
    img = Image.open(img_path)
    img.convert("L").save(new_img_path, dpi=(300, 300))
    with open(gt_path, "w", encoding="utf-8") as f:
        f.write(word)
print("start box generate")
for i, row in df.iterrows():
    filename_base = os.path.splitext(os.path.basename(row["new_path"]))[0]
    tif_path = os.path.join(output_dir, filename_base + ".tif")

    if not os.path.exists(tif_path):
        continue

    try:
        subprocess.run([
            "tesseract", tif_path, tif_path[:-4], "-l", "rus", "--psm", "13", "makebox"
        ], check=True)
    except subprocess.CalledProcessError:
        print(f"Ошибка при создании .box для {tif_path}")
    if i % 100 == 0:
        print(i)

start box generate
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


In [7]:
!rm -r /content/output /content/tessdata_best/
!mkdir /content/tessdata_best
!wget -O /content/tessdata_best/rus.traineddata https://github.com/tesseract-ocr/tessdata_best/raw/main/rus.traineddata
%cd /content/tesstrain

with open("alphabet.txt", "w", encoding="utf-8") as f:
    f.write("а б в г д е ё ж з и й к л м н о п р с т у ф х ц ч ш щ ъ ы ь э ю я - 0 1 2 3 4 5 7 8 9")

!make training \
    MODEL_NAME=rus_custom \
    START_MODEL=rus \
    GROUND_TRUTH_DIR=/content/data/part_1_gt \
    CHARACTERS_FILE=/content/tesstrain/alphabet.txt \
    MAX_ITERATIONS=500 \
    TESSDATA=../tessdata_best \
    TESSDATA_REPO=../tessdata_best \
    LANG_TYPE= \
    NORM_MODE=1 \
    NET_SPEC="[1,36,0,1 Ct3,3,16 Mp3,3 Lfys48 Lfx96 Lrx96 Lfx256 O1c]" \
    OUTPUT_DIR=data/rus_custom \
    NORM_MODE=1 \
    QUIET=1


Streaming output truncated to the last 5000 lines.
Page 1
tesseract "/content/data/part_1_gt/haditakis_8715.tif" /content/data/part_1_gt/haditakis_8715 --psm 13 lstm.train
Tesseract Open Source OCR Engine v4.1.1 with Leptonica
Page 1
tesseract "/content/data/part_1_gt/dar_ja_8716.tif" /content/data/part_1_gt/dar_ja_8716 --psm 13 lstm.train
Tesseract Open Source OCR Engine v4.1.1 with Leptonica
Page 1
tesseract "/content/data/part_1_gt/kontrabas_8717.tif" /content/data/part_1_gt/kontrabas_8717 --psm 13 lstm.train
Tesseract Open Source OCR Engine v4.1.1 with Leptonica
Page 1
tesseract "/content/data/part_1_gt/gorohova_8718.tif" /content/data/part_1_gt/gorohova_8718 --psm 13 lstm.train
Tesseract Open Source OCR Engine v4.1.1 with Leptonica
Page 1
tesseract "/content/data/part_1_gt/svetlana_8719.tif" /content/data/part_1_gt/svetlana_8719 --psm 13 lstm.train
Tesseract Open Source OCR Engine v4.1.1 with Leptonica
Page 1
tesseract "/content/data/part_1_gt/sirius_8720.tif" /content/data/part_1

In [8]:
!sudo cp /content/tesstrain/data/rus_custom.traineddata /usr/share/tesseract-ocr/4.00/tessdata/
!tesseract /content/data/part_1/eh/ehbeubaba_16235.jpg stdout \
  --tessdata-dir /usr/share/tesseract-ocr/4.00/tessdata/ \
  --oem 1 --psm 7 -l rus_custom

Failed to load any lstm-specific dictionaries for lang rus_custom!!
ЭБЕУБАБА


## :))

In [11]:
import pandas as pd
import subprocess
from tqdm import tqdm
import re

df = pd.read_csv("/content/data/part_1/all_files_final.csv").head(10000)
def tesseract_predict(img_path):
    try:
        result = subprocess.run([
            "tesseract", img_path, "stdout",
            "--tessdata-dir", "/usr/share/tesseract-ocr/4.00/tessdata/",
            "--oem", "1",
            "--psm", "7",
            "-l", "rus_custom"
        ], capture_output=True, text=True, check=True)

        pred = result.stdout.strip().lower()
        # Оставляем только русские буквы и цифры
        pred = re.sub(r"[^а-яё0-9\-]", "", pred)
        return pred

    except subprocess.CalledProcessError as e:
        print(f"⚠️ Ошибка на {img_path}")
        return ""
results = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    img_path = row["new_path"]
    true_word = str(row["word_true"]).strip().lower()

    pred = tesseract_predict(img_path)

    results.append({
        "id": row["id"],
        "true": true_word,
        "pred": pred,
        "match": pred == true_word
    })

df_results = pd.DataFrame(results)
df_results.to_csv("tesseract_eval_results.csv", index=False)

accuracy = df_results["match"].mean()
print(f"🎯 Точность Tesseract: {accuracy:.2%}")


100%|██████████| 10000/10000 [33:08<00:00,  5.03it/s]


🎯 Точность Tesseract: 30.10%
